In [1]:
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
from tweepy import API
import csv

import tweepy
import datetime
import xlsxwriter
import sys
import emoji
import re

In [2]:
def unicode_to_ascii(text):
    clean_text = (text.replace(u"\xe2\x80\x99","'").
            replace(u"\xc3\xa9", "e").
            replace(u"\xe2\x80\x90", '-').replace(u"\xe2\x80\x91", '-').
            replace(u"\xe2\x80\x92", '-').replace(u"\xe2\x80\x93", '-').
            replace(u"\xe2\x80\x94", '-').replace(u"\xe2\x80\x94", '-').
            replace(u"\xe2\x80\x98", "'").replace(u"\xe2\x80\x9b", "'").
            replace(u"\xe2\x80\x9c", '"').replace(u"\xe2\x80\x9c", '"').
            replace(u"\xe2\x80\x9d", '"').replace(u"\xe2\x80\x9e", '"').
            replace(u"\xe2\x80\x9f", '"').replace(u"\xe2\x80\xa6", '...').
            replace(u"\xe2\x80\xb2", "'").replace(u"\xe2\x80\xb3", "'").
            replace(u"\xe2\x80\xb4", "'").replace(u"\xe2\x80\xb5", "'").
            replace(u"\xe2\x80\xb6", "'").replace(u"\xe2\x80\xb7", "'").
            replace(u"\xe2\x81\xba", "+").replace(u"\xe2\x81\xbb", "-").
            replace(u"\xe2\x81\xbc", "=").replace(u"\xe2\x81\xbd", "(").
            replace(u"\xe2\x81\xbe", ")")
                 )
    return clean_text

In [3]:
def tweet_emogi_clean(text):
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)
    clean_text = emoji_pattern.sub(r'', text)
    return clean_text
    

In [4]:
'''Author: yanofsky github: https://gist.github.com/yanofsky/5436496 with small modification
'''
def get_all_tweets(username):
    
    startDate = datetime.datetime(2016, 4, 20, 0, 0, 0)
    endDate =   datetime.datetime(2018, 4, 20, 0, 0, 0)
    
    #authorize twitter, initialize tweepy
    auth = OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_key, access_secret)
    api = API(auth)
    
    #initialize a list to hold all the tweepy Tweets
    alltweets = []
    
    #make initial request for most recent tweets (200 is the maximum allowed count)
    new_tweets = api.user_timeline(screen_name = username,count=200, include_rts = True)
    
    #save most recent tweets
    for tweet in new_tweets:
        tweet.text = tweet_emogi_clean(tweet.text)
        tweet.text =unicode_to_ascii(tweet.text)
        alltweets.append(tweet)
    
    #save the id of the oldest tweet less one
    oldest = alltweets[-1].id - 1
    
    #keep grabbing tweets until there are no tweets left to grab
    if len(new_tweets) > 0:
        while len(new_tweets) > 0 :
            #all subsiquent requests use the max_id param to prevent duplicates
            new_tweets = api.user_timeline(screen_name = username,count=200, max_id=oldest, include_rts = True)
            print(len(new_tweets))
            for tweet in new_tweets:
                if tweet.created_at < startDate:
                    new_tweets=[]
                else:
                    tweet.text = tweet_emogi_clean(tweet.text)
                    tweet.text =unicode_to_ascii(tweet.text)
                    alltweets.append(tweet)
            oldest = alltweets[-1].id - 1
        
        
         
    #transform the tweepy tweets into a 2D array that will populate the csv	
    outtweets = [[tweet.id_str, tweet.created_at, str(tweet.text)] for tweet in alltweets]
    print('outtweets',len(outtweets), type(outtweets) ,outtweets[0])
    
    #write the csv
    with open('player/%s_tweets.csv' % username, 'w',newline='',encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerow(["id","created_at","text"])
        writer.writerows(outtweets)
    pass

In [5]:
#This module borrows some basic tweet api code from http://adilmoujahid.com/posts/2014/07/twitter-analytics/
#https://apps.twitter.com/app/
#verification
access_key = "934999519615471617-E7taoh5G9YN3Fd9CY6SlQGE8YYtcIh4"
access_secret = "53R7isf543FB02klxHNUDUpDDaPjQbNU4WxUwLs0ej26X"
consumer_key = "wPPQ79DMujwIJMU2O2lbq6zMV"
consumer_secret = "JrAH7lKkbdHm1Wynjw9m8bssa7hweohnpvKSA6RtSnZVullp0P"


In [9]:
if __name__ == '__main__':
    with open('playerlistfull.csv', 'r') as csvfile:
        reader = csv.reader(csvfile, delimiter=' ')
        for row in reader:
            if row != '':
                print(row[0])
                try:
                    get_all_tweets(row[0])
                    print('got ' ,row , ' tweets')
                except :
                    print("Unexpected error:", sys.exc_info()[0])

kobebryant
199
outtweets 209 <class 'list'> ['987136529419005952', datetime.datetime(2018, 4, 20, 1, 11, 3), 'Ha! Next #detail episode will be the 30th. I haven’t decided which player will be the focus. https://t.co/N0zVcSZJUq']
got  ['kobebryant']  tweets
kawhileonard
0
outtweets 4 <class 'list'> ['618580493425115136', datetime.datetime(2015, 7, 8, 0, 40, 56), 'RT @GoAztecsFacOps: Had a special guest today at the @GoAztecs Jeff Jacobs #JAMCenter. Thanks for stopping by @kawhileonard ! #AztecMBB htt…']
got  ['kawhileonard']  tweets
KingJames
200
200
outtweets 493 <class 'list'> ['985157276750237697', datetime.datetime(2018, 4, 14, 14, 6, 12), 'Preach “PrimeTime”! Once u understand that you’ll be better off. #Facts #HandleYourBusiness ✊ https://t.co/YMwYp1m77d']
got  ['KingJames']  tweets
KlayThompson
200
outtweets 198 <class 'list'> ['976977517713137665', datetime.datetime(2018, 3, 23, 0, 22, 46), 'Want to meet me and have a drive on the BMW M Tracks? Here’s your chance – https://t.co/

200
outtweets 317 <class 'list'> ['986364690505744384', datetime.datetime(2018, 4, 17, 22, 4, 2), 'Encore un coup de vieux pour moi, que de voir partir une légende du rugby francais de ma génération. À bientôt pour… https://t.co/B0lyYfVadI']
got  ['theborisdiaw']  tweets
Al_Horford
200
outtweets 199 <class 'list'> ['985137266233368576', datetime.datetime(2018, 4, 14, 12, 46, 41), 'RT @NBA: #NBAPlayoffs #NBAPlayoffs #NBAPlayoffs #NBAPlayoffs #NBAPlayoffs #NBAPlayoffs #NBAPlayoffs #NBAPlayoffs #NBAPlayoffs #NBAPlayoffs…']
got  ['Al_Horford']  tweets
tiagosplitter
200
198
outtweets 529 <class 'list'> ['987489634606239744', datetime.datetime(2018, 4, 21, 0, 34, 10), ' Rogerio está de parabéns!! Precisamos de mais pessoas como ele e ninguém para nossa basquete!   https://t.co/F2UEf7Hkum']
got  ['tiagosplitter']  tweets
JaredDudley619
200
200
200
200
200
200
200
outtweets 1478 <class 'list'> ['987694107601485824', datetime.datetime(2018, 4, 21, 14, 6, 40), 'You see this @jasonrmcintyre?!! Ca

IndexError: list index out of range